# Mochila Quadrática $0$-$1$ (QKP)

## Dados

Considere dos seguintes dados:

-  $N = \{1, \ldots, n\}$ um conjunto dos objetos.

-  $P = \{p_{ij} \in \mathbb{R}_{+} : \ i \leq j, \ i, j \in N \}$ o conjunto dos benefícios aos itens de $N$ e a relação entre eles.

-  $W = \{ w_i \in \mathbb{N} : i \in N \}$ o conjunto dos pesos dos itens de $N$.

-  $c \in \mathbb{N}$ capacidade da mochila, onde $\displaystyle\max_{i \in N} w_i \leq c < \sum_{i  \in N} w_i$.

## QKP 0-1 linear

Utilizando os dados acima para definir o problema da mochila quadrática 0-1.

\begin{align}
max \; & \sum_{i \in N} q_{i} x_i + \sum_{i = 1}^{n-1} \sum_{j =i+1}^{n}
p_{ij} y_{i,j} \\
& \sum_{i \in N} w_{i} x_{i} \leq c, \\
&  y_{i,j} \leq x_{i}, \\
&  y_{i,j} \leq x_{j}, \\
&  x_{i} + x_{j} \leq y_{i,j} + 1, \\
& x_{i} \in \left\{0, 1 \right\}, \ \ i  \in N.
\end{align}

onde $q_{i} = p_{ii}, \ i \in N$.

## Modelando o problema e resolvendo com gurobipy

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [2]:
def read_instance(dim,perc,t):

    instance = f"instances/{dim}/{dim}_{perc}_{t}.txt"

    with open(instance, 'r') as file: linhas = file.readlines()

    # remove linha vazia inicial e elimina os "\n" de cada linha
    linhas = [a.strip() for a in linhas] 

    # ler o tamanho da instancia
    n = int(linhas[0]) 

    # ler a diagonal da matriz
    d = np.fromstring(linhas[1], dtype=int, sep = ' ') 

    # define a matriz
    p = np.zeros((n,n), dtype=int) 

    # preenche a diagonal
    for i in range(n): 
        p[i][i] = d[i]

    # preenche o resto da matriz
    for i in range(n-1): 
        linha = np.fromstring(linhas[i+2], dtype=int, sep = ' ')
        for j in range(n-(i+1)):
            p[i][j+i+1] = linha[j]
            p[j+i+1][i] = p[i][j+i+1]

    # ler a capacidade
    c = int(linhas[n+2]) 

    # ler os pesos
    w = np.fromstring(linhas[n+3], dtype=int, sep = ' ') 

    return n, p, w, c

In [3]:
def qknapsack_linear(dim,perc,t):

    n, p, w, c = read_instance(dim, perc, t)

    #cria o modelo
    model = gp.Model("qkp_linear") 

    x = []
    for j in range(0, n):
        x.append(model.addVar(vtype=GRB.BINARY, name="x_{}".format(j+1)))

    l = list(tuple())
    for i in range(0, n):
        for j in range(i+1, n):
            l.append((i,j))

    y = model.addVars(l, vtype=GRB.BINARY, name='y')

        
    model.Params.TimeLimit = 120
    model.Params.MIPGap = 1.e-6
    model.Params.Threads = 1
    
    # Turn off display
    gp.setParam('OutputFlag', 0)

    obj = 0 
    for i in range(0, n):
        obj += p[i][i] * x[i]
        for j in range(i+1, n):
            obj += p[i][j] * y[i,j]

    model.setObjective(obj, GRB.MAXIMIZE)

    constr = 0
    for j in range(0, n):
        constr += (w[j] * x[j])
    model.addConstr(constr <= c)

    for i in range(0,n):
        for j in range(i+1, n):
            constr1 = y[i,j]
            model.addConstr(constr1 <= x[i])

    for i in range(0,n):
        for j in range(i+1, n):
            constr2 = y[i,j]
            model.addConstr(constr2 <= x[j])

    for i in range(0,n):
        for j in range(i+1, n):
            constr3 = x[i] + x[j]
            model.addConstr(constr3 <= 1 + y[i,j])


    model.write(f"lp/qkp_linear_{dim}_{perc}_{t}.lp")

    model.optimize()
    
    status = 0
    if model.status == GRB.OPTIMAL:
        status = 1
 
    ub = model.objBound
    lb = model.objVal
    gap = model.MIPGap
    time = model.Runtime
    nodes = model.NodeCount

    model.dispose()

    #lower bound, upper bound, gap, time, nodes
    arquivo = open(f'result/qkp_linear_{dim}_{perc}.txt','a')
    arquivo.write(
        str(f"{dim}_{perc}_{t}")+';'
        +str(round(lb,1))+';'
        +str(round(ub,1))+';'
        +str(round(gap,2))+';'
        +str(round(time,2))+';'
        +str(round(nodes,1))+';'
        +str(round(status,1))+'\n'
    )
    arquivo.close()

In [4]:
# __main__ : “top-level code environment”

# What is the “top-level code environment”?
# __main__ is the name of the environment where top-level code is run. 
# “Top-level code” is the first user-specified Python module that starts running. 
# It’s “top-level” because it imports all other modules that the program needs. 
# Sometimes “top-level code” is called an entry point to the application.

if __name__ == "__main__":

    for dim in [100]:
        for perc in [25,50]:
            for t in range(1,11):
                qknapsack_linear(dim,perc,t)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-15
Set parameter TimeLimit to value 120
Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Optimize a model with 14851 rows, 5050 columns and 34750 nonzeros
Model fingerprint: 0xcc6ca6ce
Variable types: 0 continuous, 5050 integer (5050 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 26.0000000
Presolve removed 12338 rows and 3694 columns
Presolve time: 0.08s
Presolved: 2513 rows, 1356 columns, 5124 nonzeros
Variable types: 0 continuous, 1356 integer (1356 binary)
Found heuristic solution: objective 2341

## Tabelando os resultados 

In [ ]:
import pandas as pd

dim = 100
perc = 25
inst_=f'result/qkp_linear_{dim}_{perc}.csv'

df = pd.DataFrame()
df = pd.read_csv(inst_,header=None,sep=';')

tab = pd.DataFrame()
tab = pd.concat([tab, df], ignore_index=True)
tab.columns = ['instance','bs','bb','gap','time','nodes','opt']

resume = pd.DataFrame({
    'instance':f"resume",
    'bs':tab["bs"].mean(),
    'bb':tab["bb"].mean(),
    'gap':tab['gap'].mean(),
    'time':tab['time'].mean(),
    'nodes':tab['nodes'].mean(),
    'opt':tab['opt'].sum(),
     },index=[f"qkp"]
)

tab = pd.concat([tab, resume], ignore_index=True)

tab["bb"] = tab["bb"].round(2)
tab["bs"] = tab["bs"].round(2)
tab["gap"] = tab["gap"].round(2)
tab["time"] = tab["time"].round(2)
tab["nodes"] = tab["nodes"].round(2)
tab["opt"] = tab["opt"].round().astype('Int64')

tab

In [ ]:
print(
    tab[['instance','bs','bb','gap','time','nodes','opt']].
    to_latex(index=False,float_format="%.2f")
)